Self check to write a small delta table and then read it from a new spark session.

In [ ]:
result = {
    "area": "spark",
    "description": "Check that spark can use iceberg tables.",
    "passed": False,
    "message": "",
    "plugin": "spark",
}

In [ ]:
import json

from freeds.spark import get_spark_session, show_spark_info

try:
    spark = get_spark_session("self-check1")
    db_name = "selfcheck_db"
    table_name = f"{db_name}.selfcheck_tbl"

    # create some data in delta
    print(f"Dropping and recreating database {db_name}")
    spark.sql(f"DROP DATABASE IF EXISTS {db_name} CASCADE")
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Writing table {table_name}")
    data = spark.range(100)
    (
        data.write.option(  # .mode("overwrite")  # Options: 'overwrite', 'append', 'ignore', 'error' (default)
            "mergeSchema", "true"
        )
        .format("iceberg")  # Options: 'parquet', 'csv', 'json', 'orc', 'iceberg', 'delta' etc.
        .saveAsTable(table_name)
    )
    spark.stop()

    # read some data in delta
    spark = get_spark_session("self-check2")
    data = spark.table(table_name)
    show_spark_info(spark)
    data.show(5)

    # clean up
    spark.sql(f"DROP DATABASE IF EXISTS {db_name} CASCADE")
    spark.stop()

    result["message"] = "Executed spark cell ok."
    result["passed"] = True

except Exception as ex:
    result["message"] = str(ex)
    print(str(ex))
    result["passed"] = False

25/09/13 18:16:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


NameError: name 'result' is not defined

In [ ]:
print(json.dumps(result, indent=4))